# Проектная работа "Исследование надёжности заёмщиков"

***Описание проекта***

Клиентом является кредитный отдел банка, исследование направлено на выяснение влияния семейного положения и количества детей клиента на своевременное погашение кредита. Входные данные предоставлены банком в виде статистики о финансовой способности клиентов. Результаты исследования будут учтены при разработке модели кредитного скоринга. Эта модель представляет собой специальную систему, оценивающую способность потенциального заёмщика возвратить кредит банку.

***Описание данных***

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

## Изучим данные

In [ ]:
import pandas as pd # импортируем библиотеку pandas
from IPython.display import display

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practicum_project/Проекты_Яндекс_Практикум/Исследование надежности заемщиков/Исследование надежности заемщиков.csv') # прочитаем csv-файл

In [ ]:
print(data.head(20))# посмотрим первые 20 строк данных

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [ ]:
data.info() # выведем информацию о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В датафрейме выявили пропущенные значения в столбцах days_employed и  total_income. Выведем количество пропущенных значений для каждого столбца.

In [ ]:
data.isna().sum() # количество пропущенных значений


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

## Проведем предобработку данных

### Заполним пропуски

Начнем с рассмотрения столбца total_income, который содержит информацию о доходах. Тип занятости сильно влияет на сумму дохода, поэтому мы планируем заполнить пропуски в этом столбце медианными значениями для каждого типа из столбца income_type. Например, если у человека указан тип занятости сотрудник, пропущенное значение в столбце total_income будет заполнено медианным доходом среди всех записей с тем же типом занятости.

In [ ]:
# перебираем каждый тип занятости в наборе уникальных значений столба income_type
for type in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (type) и есть пропуски в total_income
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type), 'total_income'].median()
    # и записываем в них среднее значение total_income среди строк с текущим типом занятости (type)

# проверяем, что все пропуски заполнены
print(data['total_income'].isna().sum())


0


Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [ ]:
# перебираем каждый тип занятости в наборе уникальных значений столба income_type
for type in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (type) и есть пропуски в days_employed
    data.loc[(data['income_type'] == type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == type), 'days_employed'].median()
    # и записываем в них медианное значение days_employed среди строк с текущим типом занятости (type)

# проверяем, что все пропуски заполнены
print(data['days_employed'].isna().sum())

0


Убедимся, что все пропуски заполнены. Выведем количество пропущенных значений для каждого столбца.

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработаем аномальные значения

В данном случае аномалия это присутствие отрицательного количества дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [ ]:
data['days_employed'] = data['days_employed'].abs() # произведем замену
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
days_employed_mediana = data.groupby('income_type')['days_employed'].median()# ваш код здесь
print(days_employed_mediana)

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


Аномально большие значения у безработных и пенсионеров оставляем, так как их очень сложно исправить, без доступа к методике сбора и фиксации этих данных.

Выведем перечень уникальных значений столбца `children`.

In [ ]:
children_column = data['children'].unique()# ваш код здесь
print(children_column)

[ 1  0  3  2 -1  4 20  5]


Удалим строки, в столбце `children` где встречаются аномальные значения.

In [ ]:
# воспользуемся оператором drop
data.drop(data[data['children'] == -1].index, inplace=True)
data.drop(data[data['children'] == 20].index, inplace=True)


Выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [ ]:
children_column = data['children'].unique()
print(children_column)

[1 0 3 2 4 5]


### Изменение типов данных

Изменим тип данных.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Проведем обработку неявных дубликатов в столбце `education`. Приведем к нижнему регистру все значения.

In [ ]:
data['education'] = data['education'].str.lower()

Избавимся от дубликатов строк

In [ ]:
duplicates = data.duplicated() # считаем количество дубликатов
print(duplicates.sum())

71


In [ ]:
data = data.drop_duplicates() # удалим дубликаты
duplicates = data.duplicated() # снова посчитаем количество дубликатов
print(duplicates.sum()) # выведем результат

0


### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [ ]:
def categorize_income(total_income):  # создаем функцию categorize_income()
# если доход клиента до 30000 включительно то относим его к категории 'E'
    if total_income <= 30000:
        return 'E'
# если доход клиента находится в диапазоне 30001–50000 то относим его к категории 'D'
    if 30001 <= total_income <= 50000:
        return 'D'
# если доход клиента находится в диапазоне 50001–200000 то относим его к категории 'C'
    if 50001 <= total_income <= 200000:
        return 'C'
# если доход клиента находится в диапазоне 200001–1000000 то относим его к категории 'B'
    if 200001 <= total_income <= 1000000:
        return 'B'
# если доход клиента составляет 1000001 и выше то относим его к категории 'A'
    if total_income >= 1000001:
        return 'A'



In [ ]:
# применим функцию методом apply()
data['total_income_category'] = data['total_income'].apply(categorize_income)


Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Таким образом, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` появиться строка `'операции с автомобилем'`.

In [ ]:
def categorize_purpose(purpose):  # создадим функцию categorize_purpose()
# если в столбце 'purpose' содержится 'жил' или 'недвиж', тогда относим цель к категории 'операции с недвижимостью'
    if ('жил' in purpose) or ('недвиж' in purpose):
        return 'операции с недвижимостью'
# если в столбце 'purpose' содержится 'авто', тогда относим цель к категории 'операции с автомобилем'
    if 'авто' in purpose:
        return 'операции с автомобилем'
# если в столбце 'purpose' содержится 'свадьб', тогда относим цель к категории 'проведение свадьбы'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
# если в столбце 'purpose' содержится 'образован', тогда относим цель к категории 'получение образования'
    if 'образован' in purpose:
        return 'получение образования'


In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose) # примените функцию

In [ ]:
# посмотрим на получившиеся цели кредитов
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

## Исследовательский анализ данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
from IPython.display import display # импортируем display
# сделаем группировку по колонке 'children', посчитаем сумму эпизодов просрочки по каждой группе
grouped_1 = data.groupby('children')['debt'].sum()

print('ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В КАЖДОЙ ИЗ ГРУПП С ОПРЕДЕЛЕННЫМ КОЛИЧЕСТВОМ ДЕТЕЙ')

# убирем служебное описание данных
grouped_1.name = None
print(grouped_1.to_string(header=False))

print() # выведем пустую строку для удобного чтения результатов

# сделаем группировку по колонке 'children', посчитаем количество записей в таблице по каждой группе
grouped_2 = data.groupby('children')['debt'].count()

print('ОБЩЕЕ КОЛИЧЕСТВО ЗАЕМЩИКОВ В КАЖДОЙ ИЗ ГРУПП С ОПРЕДЕЛЕННЫМ КОЛИЧЕСТВОМ ДЕТЕЙ')

print(grouped_2.to_string(header=False))

print() # выведем пустую строку для удобного чтения результатов

# рассчитаем процент. Оставим 4 знака после запятой, а так же отсортируем результат по убыванию
result = ((grouped_1 / grouped_2).round(4)).sort_values(ascending=False)

formatted_result = result.apply(lambda x: '{:.2f}%'.format(x*100)) # выведем результат в процентах с двумя знаками после запятой

print('ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ КОЛИЧЕСТВА ДЕТЕЙ')
print(formatted_result)

# выведем объединенную таблицу
combined_grouping = data.groupby('children').agg({'debt': ['sum','count']})
display(combined_grouping)

ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В КАЖДОЙ ИЗ ГРУПП С ОПРЕДЕЛЕННЫМ КОЛИЧЕСТВОМ ДЕТЕЙ
0    1063
1     444
2     194
3      27
4       4
5       0

ОБЩЕЕ КОЛИЧЕСТВО ЗАЕМЩИКОВ В КАЖДОЙ ИЗ ГРУПП С ОПРЕДЕЛЕННЫМ КОЛИЧЕСТВОМ ДЕТЕЙ
0    14091
1     4808
2     2052
3      330
4       41
5        9

ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ КОЛИЧЕСТВА ДЕТЕЙ
children
4    9.76%
2    9.45%
1    9.23%
3    8.18%
0    7.54%
5    0.00%
dtype: object


debt       
           sum  count
children             
0         1063  14091
1          444   4808
2          194   2052
3           27    330
4            4     41
5            0      9

**Вывод:** Полученные данные показывают анамалию в значении показателя у родителей с 5 детьми. Совсем нет просрочки.Такой результат, возможно, связан с тем, что они брали небольшой кредит и имели достаточный доход для его погашения. Но, поскольку таких семей в выборке всего 9 то, получившийся результат нельзя будет использовать для выводов.  Лучше для данной категории использовать медианное значение процента просрочки от всех не нулевых результатов других категорий вместо полученного значения 0%. Рассматривая остальные величины, мы видим, что минимальное значение просрочки у людей без детей. Затем идет показатель просрочки у семей с 3 детьми. У оставшихся категорий прослеживается закономерность: чем больше детей, тем чаще просрочка.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# выполним группировку по колонке 'family_status', посчитаем сумму эпизодов просрочки по каждой группе
grouped_3 = data.groupby('family_status')['debt'].sum()

print('ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В ЗАВИСИМОСТИ ОТ СЕМЕЙНОГО ПОЛОЖЕНИЯ')

grouped_3.name = None # уберем служебное описание данных
print(grouped_3.to_string(header=False))

print() # напечатаем пустую строку для удобного чтения результатов

# сделаем группировку по колонке 'family_status', посчитаем количество соответствующих заемщиков в данных
grouped_4 = data.groupby('family_status')['debt'].count()

print('КОЛИЧЕСТВО ЗАЕМЩИКОВ С СООТВЕТСТВУЮЩИМ СЕМЕЙНЫМ СТАТУСОМ В ПРЕДСТАВЛЕННЫХ ДАННЫХ')

print(grouped_4.to_string(header=False))
print() # напечатаем пустую строку для удобного чтения результатов

# рассчитываем процент. Оставим 4 знака после запятой, а так же отсортируем результат по убыванию
result_1 = ((grouped_3 / grouped_4).round(4)).sort_values(ascending=False)

formatted_result_1 = result_1.apply(lambda x: '{:.2f}%'.format(x*100)) # выводим результат в процентах с двумя знаками после запятой
print('ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ СЕМЕЙНОГО СТАТУСА')

print(formatted_result_1.to_string(header=False))

# выведем объединенную таблицу
combined_grouping = data.groupby('family_status').agg({'debt': ['sum','count']})
display(combined_grouping)

ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В ЗАВИСИМОСТИ ОТ СЕМЕЙНОГО ПОЛОЖЕНИЯ
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927

КОЛИЧЕСТВО ЗАЕМЩИКОВ С СООТВЕТСТВУЮЩИМ СЕМЕЙНЫМ СТАТУСОМ В ПРЕДСТАВЛЕННЫХ ДАННЫХ
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
гражданский брак          4134
женат / замужем          12261

ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ СЕМЕЙНОГО СТАТУСА
Не женат / не замужем    9.76%
гражданский брак         9.31%
женат / замужем          7.56%
в разводе                7.06%
вдовец / вдова           6.62%


debt       
                       sum  count
family_status                    
Не женат / не замужем  273   2796
в разводе               84   1189
вдовец / вдова          63    951
гражданский брак       385   4134
женат / замужем        927  12261

**Вывод:** В данной выборке максимальный риск просрочки показали клиенты с семейным статусом 'Не женат / не замужем' . Минимальный выявлен у вдовцов.
По результатам можно представить 2 новые категории: люди, которые официально докуминтировали свой статус: женат / замужем, в разводе, вдовец / вдова. И остальные: Не женат / не замужем, гражданский брак. И внутри этих категорий прослеживается тенденция следующего свойства: чем более глубокий опыт семейной жизни имеется у клиентов, тем больше их ответственность в финансовом плане.





Нужно отметить, что в категории желательно должно быть не менее 5% от числа выборки. Тогда это 20000*0,05= 1000 человек. Если воспользоваться таким подходом, то можно говорить о том, что выводы о возврате кредита в срок можно делать только по 2 категориям: 'C' с доходом 50001–200000 и 'B' с доходом 200001–1000000. Чтобы осуществить корректные выводы по остальным категориям нам потребуется собрать данные за больший промежуток времени.
Пока мы можем констатировать, что чем выше доход, тем выше вероятность возврата кредита в срок.

### Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# выполним группировку по колонке 'purpose_category', посчитаем сумму эпизодов просрочки по каждой группе
grouped_7 = data.groupby('purpose_category')['debt'].sum()

print('ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В ЗАВИСИМОСТИ ОТ ЦЕЛИ КРЕДИТА')

grouped_7.name = None # убираем служебное описание данных

print(grouped_7.to_string(header=False))

print() # напечатаем пустую строку для удобного чтения результатов

# количество соответствующих заемщиков в данных
grouped_8 = data.groupby('purpose_category')['debt'].count()

print('КОЛИЧЕСТВО ЗАЕМЩИКОВ С СООТВЕТСТВУЮЩЕЙ ЦЕЛЬЮ В ПРЕДСТАВЛЕННЫХ ДАННЫХ')

print(grouped_8.to_string(header=False))

print()

# рассчитаем процент. Оставим 4 знака после запятой, а так же отсортируем результат по убыванию
result_3 = ((grouped_7 / grouped_8).round(4)).sort_values(ascending=False)

formatted_result_3 = result_3.apply(lambda x: '{:.2f}%'.format(x*100))# выведем результат в процентах с двумя знаками после запятой

print('ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ ЦЕЛИ КРЕДИТА')
print(formatted_result_3.to_string(header=False))

ОБЩЕЕ КОЛИЧЕСТВО СЛУЧАЕВ ПРОСРОЧКИ КРЕДИТА В ЗАВИСИМОСТИ ОТ ЦЕЛИ КРЕДИТА
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183

КОЛИЧЕСТВО ЗАЕМЩИКОВ С СООТВЕТСТВУЮЩЕЙ ЦЕЛЬЮ В ПРЕДСТАВЛЕННЫХ ДАННЫХ
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313

ПРОЦЕНТ ПРОСРОЧКИ ВОЗВРАТА КРЕДИТА В ЗАВИСИМОСТИ ОТ ЦЕЛИ КРЕДИТА
операции с автомобилем      9.35%
получение образования       9.25%
проведение свадьбы          7.91%
операции с недвижимостью    7.26%


**Вывод:** Наиболее ответственными в части своевременной оплаты кредита оказались люди, бравшие кредит, связанный с операциями по недвижимости. Наименее ответственными оказались люди, бравшие кредит на операции с автомобилем.

## Общий вывод.

Первичная обработка данных

Первичное изучение предоставленных данных выявило несколько недостатков, часть из которых требовала устранения.

в столбцах «days_employed», «total_income» были пропуски
в столбце «days_employed» встречаются артефакты — значения, которые не отражают действительность и появились по какой-то ошибке. В частности, отрицательное количество дней трудового стажа или аномально большие значения дней трудового стажа для категорий безработный и пенсионер.
в столбце children есть два аномальных значения: -1 и 20 детей.
в столбце education содержатся неявные дубликаты - значения, записанные по-разному. С использованием заглавных и строчных букв. В итоге, для получения корректных результатов исследования, на этапе предобработки были выполнены следующие действия:
в столбце total_income пропуски заполнили медианным значением по каждому типу из столбца. Медиану взяли чтобы не влияли выбросы.
в столбце days_employed все отрицательные значения заменены положительными
в столбце children аномальные значения удалили вместе с содержащими их строками
в столбце education выявили и удалили неявные дубликаты Для удобного представления и анализа информации была проведена категоризация целей, на которые берется кредит:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования' А также, клиенты были разделены на категории по доходу:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Результаты

В результате проведенного исследования выяснены следующие закономерности:

- чем больше детей, тем чаще просрочка;
- люди, когда-либо бывавшие в браке, реже остаются в долгу по кредиту. Это касается как людей в разводе, так и людей, находящихся в браке сейчас.
- чем выше доход, тем выше вероятность возврата кредита в срок.
- наиболее ответственными в части своевременной оплаты кредита оказались люди, бравшие кредит, связанный с операциями по недвижимости. Далее расположились такие цели: проведение свадьбы, получение образования. И наименее ответственными оказались люди, бравшие кредит на операции с автомобилем.
Портреты хорошего и плохого заемщика:

Максимально надежным заемщиком для банка будет человек без детей, вдовец/вдова, имеющий доход высокий доход (200001–1000000), который берет кредит на операции с недвижимостью. В зоне максимального риска заемщики с детьми, которые указали семейный статус как «Не женат / не замужем» или живущие в гражданском браке, имеющие доход ниже или равный 30000, а также имеющие целью кредита операции с автомобилем. Рекомендации по сбору данных и скорингу: Для того чтобы результаты исследования были более точны, необходимо по возможности внести корректировки в методику сбора и фиксации данных, чтобы избежать пропущенных значений и искажения. Сравнивая такие характеристики как доход и количество детей, мы столкнулись с тем, что количество данных в категориях недостаточно для однозначного трактования. По возможности, желательно их увеличить.